# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns

###Praticando o exercicio feito pela Hashtag

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Nosso objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### O que temos disponível, inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

Elas estão disponíveis para download abaixo da aula (se você puxar os dados direto do Kaggle pode ser que encontre resultados diferentes dos meus, afinal as bases de dados podem ter sido atualizadas).

Caso queira uma outra solução, podemos olhar como referência a solução do usuário Allan Bruno do kaggle no Notebook: https://www.kaggle.com/allanbruno/helping-regular-people-price-listings-on-airbnb

Você vai perceber semelhanças entre a solução que vamos desenvolver aqui e a dele, mas também algumas diferenças significativas no processo de construção do projeto.

- As bases de dados são os preços dos imóveis obtidos e suas respectivas características em cada mês.
- Os preços são dados em reais (R$)
- Temos bases de abril de 2018 a maio de 2020, com exceção de junho de 2018 que não possui base de dados

### Expectativas Iniciais

- Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
- A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
- Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

### Importar Bibliotecas e Bases de Dados

In [1]:
import pandas as pd
import pathlib    # permite identificar uma pasta e percorrer todas as pastas


In [2]:
# #entro na pasta a partir desse caminho
# caminho_bases = pathlib.Path("dataset")  #dataset eh o nome da pasta/arquivo

# # traz todos os arquivos
# for arquivo in caminho_bases.iterdir():  #interdir -> da uma lista q pode iterar sobre ela com todos os caminhos do path
#     print(arquivo.name)
    
# # importando 1 arquivo
# abril2018_df = pd.read_csv(r'dataset\abril2018.csv')
# display(abril2018_df) # mostrara todos os arquivos q estao dentro da pasta

# # esse aviso diz q a coluna 62 e 87 tem 2 tipos de dados distintos - o q faz um pouco mais lento.

In [3]:
# # Como sao varios arquivos, cd arquivo sera adicionado embaixo um do outro (mergir) -> append

# caminho_bases = pathlib.Path("dataset") 

# base_airbnb = pd.DataFrame()            #2 criar um df vazio

# for arquivo in caminho_bases.iterdir():  
#     df = pd.read_csv(caminho_bases / arquivo.name)          # "ou df = pd.read_csv(r'dataset\{}'.format(arquivo.name)')"
#     base_airbnb = base_airbnb.append(df)

#                 #dataframe = dataframe.append(df2)  #1 criar um df q neste sera adicionado cd arquivo embaixo um do outro
    
# display(base_airbnb)

In [ ]:
# Sera adicionado na tabela mes e ano q eram identificadas pelo nome dos arquvios

# Criar um dicionario

meses = {"jan":1, "fev":2, "mar":3, "abr":4, "mai":5, "jun":6, "jul": 7, "ago":8, "set":9, "out":10, "nov":11, "dez":12}


caminho_bases = pathlib.Path("dataset") 

base_airbnb = pd.DataFrame()            

for arquivo in caminho_bases.iterdir():  
    nome_mes = arquivo.name[:3]    # pega o nome do 1o arquivo inicionando pelo abr (abril)
    mes = meses[nome_mes]
    
    ano = arquivo.name[-8:]             # usa o final d cada arquivo. ex: abril2018.csv
    ano = int(ano.replace(".csv", ""))  # substitiuindo o ".csv" por vazio""
    
    df = pd.read_csv(caminho_bases / arquivo.name)          
    df["ano"] = ano
    df["mes"] = mes
    base_airbnb = base_airbnb.append(df)

               
display(base_airbnb)

C:\Users\Keylla\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (62,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Keylla\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Keylla\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Consolidar Base de Dados

- Como ha mtas colunas, isso interferira na velocidade dessa analise. Atraves de uma breve analise  eh possivel perceber que ha mtas colunas desnecessarias que nao agregarao valor para o modelo de previsao. 
- Sendo assim, a ideia eh excluir essas colunas que nao fazem sentido ficar.
- Colunas a serem excluidas:

    1. Indice, ID, 
    2.Colunas repetidas ou extremamente similar. last_scraped (sera excluida pq ja ha 2 colunas no final de mes e ano)
    3.Colunas preenchidas com texto livre. Nao rodar Name, summary, 
    4.  
- Nesse caso criarei um arquvio em excel com os mil primeiros registros (p/aumentar a velocidade da manipulacao) e em seguida fazer uma analise qualitativaa

In [ ]:
# Imprimindo as colunas desse banco de dados 
print(list(base_airbnb.columns))


In [ ]:
# Gerando os mil primeiros registros e joga-los no excel q sera criado
# base_airbnb.head(1000).to_csv("primeiros_registros.csv", sep=",") # sep eh o separador de virgulas pq as vezes os
#dados podem ir para o excel em formato de texto

base_airbnb.head(1000).to_csv("primeiros_registros.csv")

In [ ]:
# Proximo passo eh abrir o excel e olhar toda a base de dados e comecar a deletar as colunas no proprio excel

### Se tivermos muitas colunas, já vamos identificar quais colunas podemos excluir

In [ ]:
print(base_airbnb)

### Tratar Valores Faltando

### Verificar Tipos de Dados em cada coluna

### Análise Exploratória e Tratar Outliers

### Encoding

### Modelo de Previsão

### Análise do Melhor Modelo

### Ajustes e Melhorias no Melhor Modelo